# Two stage miller OTA Sizing Script
Reference: 
- CMOS Analog Circuit Design, Phillip E. Allen
- gm/ID, Boris Murman

In [1]:
from pygmid import Lookup as lk
import numpy as np

In [2]:
#Specs
gain_bw = 10e6 #minimum gain bandwidth
fc = gain_bw
gain_db = 20 #minimum gain 20 dB
phase_margin = 90 #minimum phase margin 90 deg
c_load = 30e-12 # load capacitance
vdd = 3.3
vss = 0
vdsat_pmos = 0 #vds saturation source follower PMOS output stage
vdsat_nmos = 0 #vds saturation tail current source
v_swing = vdd + abs(vss) - vdsat_pmos - vdsat_nmos
ts = 5 / (2 * np.pi * gain_bw)
sr = v_swing / ts
c_miller = 0.22 * c_load #C Miller Compensation
i_tail = sr * c_miller #I tail
gm_1 = fc * 2 * np.pi * c_miller #input stage transconductace

print(f"Voltage Swing: {v_swing} V")
print(f"Settling Time: {round(ts*1e6,2)} us")
print(f"Slew Rate: {round(sr/1e6,2)} V/us")
print(f"C Miller: {round(c_miller*1e12,2)} pF")
print(f"Tail current: {round(i_tail*1e6,2)} uA")
print(f"gm 1: {round(gm_1*1000,2)} mS")

Voltage Swing: 3.3 V
Settling Time: 0.08 us
Slew Rate: 41.47 V/us
C Miller: 6.6 pF
Tail current: 273.7 uA
gm 1: 0.41 mS


In [3]:
def get_w(mos_type, gm, gm_id_spec, L_spec, vds, title='specs'):
    
    #Define the MOS type
    if mos_type == 0:
        mos_file = 'nfet_03v3.mat'
    elif mos_type == 1:
        mos_file = 'pfet_03v3.mat'
    else:
        print(f"MOS type: 0: NMOS 3.3V, 1: PMOS 3.3")
        return
    print(f"==Transistor: {title}==")
    print(f"gm: {round(gm*1e3,3)} mS")
    # the gm_gds we look up and calculate gds from it
    mos = lk(mos_file)
    vgs = mos.look_upVGS(GM_ID=gm_id_spec, L=L_spec, VDS=vds, VSB=0.0)
    vgs = mos.look_upVGS(GM_ID=gm_id_spec, L=L_spec, VDS=vgs, VSB=0.0)
    print('V_GS =', round(float(vgs), 3), 'V')

    # the gm_gds we look up and calculate gds from it
    gm_gds = mos.lookup('GM_GDS', GM_ID=gm_id_spec, L=L_spec, VDS=vgs, VSB=0)
    gds = gm / gm_gds
    print('gds =', round(gds/1e-9, 2), 'nS')

    # find f_T (which is not stored directly, but we can find the gm to gate capacitance ratio)
    gm_cgg = mos.lookup('GM_CGG', GM_ID=gm_id_spec, L=L_spec, VDS=vgs, VSB=0)
    f_T = gm_cgg / (2*np.pi)
    print('Cgg =', round(gm/gm_cgg/1e-15, 1), 'fF')
    print('f_T =', round(f_T/1e6, 1), 'MHz')

    # find the W of the diode transistor
    id_spec = gm/gm_id_spec
    print('Id =', round(id_spec*1000000, 2), 'uA, rounded Id =', round(id_spec*2000000)/2, 'uA')
    id_w = mos.lookup('ID_W', GM_ID=gm_id_spec, L=L_spec, VDS=vgs, VSB=0)
    w = id_spec / id_w
    print('W =', round(w, 2), 'um, rounded W =', round(w*2)/2, 'um')      

In [4]:
#Transistor sizing
length = 1 # L = 2 um
vds = vdd/2 
#Input Pairs
gmid_1 = 10
id_1 = gm_1/gmid_1
get_w(0,gm_1,gmid_1,length,vds,"M1 & M2")

#Active Load
gmid_2 = 5
gm_2 = gmid_2 * id_1
get_w(1,gm_2,gmid_2,length,vds,"M3 & M4")

#Tail Current
gmid_5 = 5
gm_5 = gmid_5 * id_1 * 2
get_w(0,gm_5,gmid_5,length,vds,"M5")

#Common Source PMOS Output Stage
gmid_6 = 10
gm_6 = 10 * gm_1
get_w(1, gm_6, gmid_6, length, vds, "M6")

#Current source NMOS Output Stage
gmid_7 = 5
id_7 = 414.69e-6
gm_7 = gmid_7 * id_7
get_w(0, gm_7, gmid_7, length, vds, "M7")

#Nulling Resistor
rz = (c_miller + c_load)/c_miller/gm_6
print(f"Rz: {rz}")

==Transistor: M1 & M2==
gm: 0.415 mS
V_GS = 0.834 V
gds = 1350.42 nS
Cgg = 56.7 fF
f_T = 1164.1 MHz
Id = 41.47 uA, rounded Id = 41.5 uA
W = 18.83 um, rounded W = 19.0 um
==Transistor: M3 & M4==
gm: 0.207 mS
V_GS = 1.148 V
gds = 819.0 nS
Cgg = 67.6 fF
f_T = 488.0 MHz
Id = 41.47 uA, rounded Id = 41.5 uA
W = 18.86 um, rounded W = 19.0 um
==Transistor: M5==
gm: 0.415 mS
V_GS = 1.038 V
gds = 1738.66 nS
Cgg = 29.6 fF
f_T = 2229.0 MHz
Id = 82.94 uA, rounded Id = 83.0 uA
W = 8.92 um, rounded W = 9.0 um
==Transistor: M6==
gm: 4.147 mS
V_GS = 0.942 V
gds = 9628.05 nS
Cgg = 2803.9 fF
f_T = 235.4 MHz
Id = 414.69 uA, rounded Id = 414.5 uA
W = 803.52 um, rounded W = 803.5 um
==Transistor: M7==
gm: 2.073 mS
V_GS = 1.038 V
gds = 8693.32 nS
Cgg = 148.0 fF
f_T = 2229.0 MHz
Id = 414.69 uA, rounded Id = 414.5 uA
W = 44.62 um, rounded W = 44.5 um
Rz: 1337.2522766674404


In [5]:
#Open Loop Gain
gm_in = 0.415e-3
gm_out = 4.147e-3
gds_in1 = 1350.42e-9
gds_in2 = 819e-9
gds_out1 = 9628.05e-9
gds_out2 = 8693.32e-9

DC_gain = gm_in*gm_out/((gds_in1+gds_in2)*(gds_out1+gds_out2))
print(f"DC gain = {np.log10(DC_gain)*20} dB")

DC gain = 92.72961094146113 dB


In [ ]:
#NMOS Input Diffpairs
get_w(0,0.4398e-3,10,2,3.3/2,'GM')

V_GS = 0.818 V
gds = 923.92 nS
Cgg = 250.4 fF
f_T = 279.6 MHz
Id = 43.98 uA, rounded Id = 44.0 uA
42.711395498628335
W = 42.71 um, rounded W = 42.5 um


In [4]:
#PMOS Active Load
gm_id = 5
id = 44e-6
gm = gm_id*id
get_w(1,gm,5,2,3.3/2)

V_GS = 1.144 V
gds = 431.94 nS
Cgg = 293.7 fF
f_T = 119.2 MHz
Id = 44.0 uA, rounded Id = 44.0 uA
43.41722214305814
W = 43.42 um, rounded W = 43.5 um


In [5]:
#NMOS Tail Current Source
gm_id = 5
id = 2*44e-6
gm = gm_id*id
get_w(0,gm,5,2,3.3/2)

V_GS = 1.033 V
gds = 1156.61 nS
Cgg = 120.0 fF
f_T = 583.7 MHz
Id = 88.0 uA, rounded Id = 88.0 uA
18.829940763124256
W = 18.83 um, rounded W = 19.0 um


In [6]:
#PMOS Output Stage
gm_id = 10
gm = 0.44e-3*10
get_w(1,gm,10,2,3.3/2)

V_GS = 0.936 V
gds = 5204.14 nS
Cgg = 12170.9 fF
f_T = 57.5 MHz
Id = 440.0 uA, rounded Id = 440.0 uA
1877.261814380366
W = 1877.26 um, rounded W = 1877.5 um


In [9]:
#NMOS Ouput Stage
gm_id = 5
id = 440e-6
gm = gm_id*id
get_w(0,gm,5,2,3.3/2)

V_GS = 1.033 V
gds = 5783.03 nS
Cgg = 599.9 fF
f_T = 583.7 MHz
Id = 440.0 uA, rounded Id = 440.0 uA
94.14970381562128
W = 94.15 um, rounded W = 94.0 um


In [4]:
#Op-Amp Gain
gm_in = 0.4398e-3
gm_out = 0.4398e-3*10
gds_in1 = 923.92e-9
gds_in2 = 431.94e-9
gds_out1 = 5204.14e-9
gds_out2 = 5783.03e-9

DC_gain = gm_in*gm_out/((gds_in1+gds_in2)*(gds_out1+gds_out2))
print(f"DC gain = {np.log10(DC_gain)*20} dB")

DC gain = 102.26819514535018 dB
